In [ ]:
# !unzip html2json.zip

In [2]:
import torch
from torch import nn
from functools import partial
from html2json import HTML_JSON_Dataset, padding_collate_fn
from torch.utils.data import DataLoader, random_split
from html2json.charactertokenizer import HTMLTokenizer, JSONTokenizer
from html2json.charactertokenizer import MASK_TOKEN
from html2json import load_data
from html2json.seq2seq import Seq2SeqTransformer
from html2json.seq2seq import translate
from html2json.training import train_epoch, evaluate
from timeit import default_timer as timer

In [3]:
torch.cuda.empty_cache()
torch.cuda.is_available()

True

In [4]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
html_pth = 'generated_tables/tables'
json_pth = 'generated_tables/metadata'

In [6]:
html_data, json_data = load_data(html_pth, json_pth, as_string=False, limit=50)

In [7]:
html_tokenizer = HTMLTokenizer(html_data)
json_tokenizer = JSONTokenizer(json_data)

C:\Workspace\CharacterBasedNMT\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
MASK_TOKEN

3

In [9]:
collate_fn = partial(padding_collate_fn, pad_token_html = MASK_TOKEN, pad_token_json = MASK_TOKEN)
html_data_str, json_data_str = load_data(html_pth, json_pth, as_string=True, limit=50)
h2j_dataset = HTML_JSON_Dataset([html_tokenizer.encode(h) for h in html_data_str], [json_tokenizer.encode(j) for j in json_data_str])
train_set, val_set = random_split(h2j_dataset, [0.8, 0.2])

In [10]:
torch.manual_seed(0)
SRC_VOCAB_SIZE = len(html_tokenizer)
TGT_VOCAB_SIZE = len(json_tokenizer)
EMB_SIZE = 256
NHEAD = 8
FFN_HID_DIM = 4096
BATCH_SIZE = 32
NUM_ENCODER_LAYERS = 1
NUM_DECODER_LAYERS = 1
LR = 0.001
NUM_EPOCHS = 1

torch.cuda.empty_cache()
train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE, collate_fn=collate_fn)
validation_dataloader = DataLoader(val_set, batch_size=BATCH_SIZE, collate_fn=collate_fn)

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=MASK_TOKEN)
optimizer = torch.optim.Adam(transformer.parameters(), lr=LR, betas=(0.9, 0.98), eps=1e-9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, threshold=0.01, threshold_mode='abs')

C:\Workspace\CharacterBasedNMT\.venv\Lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [14]:
# with profile(activities=[
        # ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True, profile_memory=True) as prof:
for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    transformer.train()
    train_loss = train_epoch(transformer, optimizer, train_dataloader, loss_fn)
    end_time = timer()
    scheduler.step(train_loss)
    # evaluation
    transformer.eval()
    val_loss = evaluate(transformer, validation_dataloader, loss_fn)
    # add save model checkpoint every 20 epochs
    if epoch % 5 == 0:
        torch.save({
                'epoch': epoch,
                'model_state_dict': transformer.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': train_loss,
                }, f"./checkpoints/checkpoint_{epoch}.pt")
    # val_loss = evaluate(transformer)
    
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s, lr: {scheduler.get_last_lr()}"))
# save the model after training
torch.save(transformer.state_dict(), "./assets/transformer.pt")

C:\Workspace\CharacterBasedNMT\.venv\Lib\site-packages\torch\nn\functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch: 1, Train loss: 4.788, Val loss: 4.303, Epoch time = 7.219s, lr: [0.001]


In [18]:
model = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, 97, 99, FFN_HID_DIM).to(DEVICE)
model.load_state_dict(torch.load("./assets/transformer.pt"))

C:\Users\Simo\AppData\Local\Temp\ipykernel_13140\3953740119.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("./assets/transformer.pt"))


<All keys matched successfully>

In [19]:
pred = translate(transformer, html_data_str[0], html_tokenizer, json_tokenizer)

C:\Workspace\CharacterBasedNMT\.venv\Lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


In [20]:
json_data_str[0]

'[{]["body"][:][{]["content"][:][[]"1060"[,]"37"[,]"1593"[,]"1364"[]][,]["headers"][:][{]["col"][:][[]"Roberts LLC"[]][,]["row"][:][[]"Daniel Brown"[,]"Shane Barnes DDS"[,]"Nicole Carpenter"[,]"Kristin Duarte"[,]"programmer"[,]"Carpenter"[,]"singer"[,]"actor"[]][}][}][,]["footer"][:][{]["table_creation_date:"][:]"3Feb2013"[,]["text"][:]"modified: 5Feb2013\\nCreation: 3Feb2013 Chad"[}][,]["header"][:][{]["table_id"][:]"59.99.9.62"[,]["text"][:]"Table 59.99.9.62 Loss adjuster, chartered"[}][}]'

In [21]:
pred

'[CLS]J3FFFFFDQ33FFFFFKB40["row"]qFS3VFS3VFFFFS3FS3FS3VFS3FKB40FFFFFFF[,]F[,]FFFFK["table_id"]/2u: :v3FK["table_id"]J3FK["row"]qFS3FS3FS3FKB4: : : :[,]FS\n:[,]R[,]FS\n:P:S3FS3\n:P:S33VFS\n:VFS3FS\nz:S\npJ36["row"]qw3\ndMz:V["table_creation_date:"]pg\n45bFS\ndM["table_creation_date:"]\ndM["table_creation_date:"]VFS\ndM["table_creation_date:"]["table_creation_date:"]["table_creation_date:"]V["table_creation_date:"]V["table_creation_date:"]["table_creation_date:"]["table_creation_date:"]["table_creation_date:"]pgf[,]vzW["table_creation_date:"]D["table_creation_date:"]D["table_creation_date:"]D["table_creation_date:"]D["table_creation_date:"]VFK["table_id"]/fsFS\ndMz:VFD2[SEP]'

In [22]:
pred[-5:5] == json_data_str[0]

False